### Imports

In [ ]:
import pandas as pd
import os
import numpy as np
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

### Load Data

In [ ]:
root_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(root_dir, 'data')
full_raw_initial_dataset_path = os.path.join(data_dir, 'gx_merged_lags_months.csv')
volume_path = os.path.join(data_dir, 'gx_volume.csv')
train_path = os.path.join(data_dir, 'train_split.csv')
features_path = os.path.join(data_dir, 'features')

In [ ]:
volume = pd.read_csv(volume_path, index_col=0)
train_ids = pd.read_csv(train_path)
full_raw_initial_dataset = pd.read_csv(full_raw_initial_dataset_path)

In [ ]:
full_initial_dataset = full_raw_initial_dataset.loc[
    full_raw_initial_dataset.test == 0,:].drop(columns = 'test').drop_duplicates()

In [ ]:
full_initial_dataset

### Functions

In [ ]:
def find_closest_volume(country, brand, month_num, length_serie, func):
    ind = (volume.country == country) & (volume.brand == brand) & (volume.month_num <month_num)
    volume_filter = volume.loc[ind, :]
    volume_sorted = volume_filter.sort_values(by=['month_num'], ascending=False)
    volume_sorted.reset_index(inplace=True, drop=True)
    total_obs = len(volume_sorted)
    total_to_select = length_serie if length_serie<=total_obs else total_obs 
    volumes_selected = volume_sorted.volume[:total_to_select].values
    return func(volumes_selected)

### Create initial datasets

In [ ]:
train = train_ids.merge(
    full_initial_dataset,
    'inner',
    on=['country', 'brand']
)

In [ ]:
#sanitiy checks
assert len(train.loc[:,['country', 'brand', 'month_num']].drop_duplicates()) == \
len(train), 'duplicated'

### Features

#### Add feature
$$vol_{-1}$$
* Name: volume_1

In [ ]:
volume_at_1 = volume.loc[volume.month_num == -1, ['country', 'brand', 'volume']].\
            drop_duplicates().\
            rename(columns={'volume':'volume_1'})

In [ ]:
full_with_volume_1 = full_initial_dataset.merge(
    volume_at_1,
    'left',
    on=['country', 'brand']
)

In [ ]:
assert len(full_initial_dataset) == len(full_with_volume_1), 'There are duplicated'

#### Add feature
$$log\Big(\frac{vol_{t} + 1}{vol_{-1}}\Big)$$
* Name: log_relative_volume

In [ ]:
train_with_relative_volume = train.merge(
    volume_at_1,
    'left',
    on=['country', 'brand']
)
train_with_relative_volume['log_relative_volume'] = np.log(
    (train_with_relative_volume.volume+1)/(train_with_relative_volume.volume_1)
)

In [ ]:
train_with_relative_volume.sort_values(by=['country', 'brand', 'month_num'], inplace=True)

In [ ]:
train_with_relative_volume['lag_log_relative_volume'] = train_with_relative_volume.groupby(
    ['country', 'brand'])['log_relative_volume'].shift(1)

In [ ]:
train_with_relative_volume['lag_log_relative_volume'] = np.where(
    train_with_relative_volume.month_num == 0,
    np.log((1+train_with_relative_volume.volume_1)/train_with_relative_volume.volume_1),
    train_with_relative_volume.lag_log_relative_volume
)

In [ ]:
features = train_with_relative_volume.drop(columns=['volume', 'log_relative_volume'])
target = train_with_relative_volume['log_relative_volume']

In [ ]:
categorical_cols = ['country', 'brand', 'therapeutic_area', 'presentation', 'month_name']
te = TargetEncoder(cols=categorical_cols)
pipe  = Pipeline([
    ("te", te),
    ("imp", SimpleImputer(strategy="mean")),
    ("sc", StandardScaler()),
    ("model", Lasso(alpha=0.001, max_iter=2000))
])

In [ ]:
pipe.fit(features, target)

In [ ]:
pipe[-1].coef_

In [ ]:
def get_log_relative_volume(model, features):
    features_copy = features.copy()
    features_copy.sort_values(by=['country', 'brand', 'month_num'], inplace=True)
    features_copy['log_relative_volume'] = float('-inf')
    i=0
    for index, row in features_copy.iterrows():
        if(i%5000 == 0):
            print('Iteration:', i)
        
        country = row.country
        brand = row.brand
        month = row.month_num
        
        if month==0:
            row.at['lag_log_relative_volume'] = 0
        else:
            ind = (features_copy.brand == brand) &\
            (features_copy.country == country) &\
            (features_copy.month_num == month-1) 
            lag_log_relative_volume = features_copy.loc[ind, 'log_relative_volume']
            row.at['lag_log_relative_volume'] = lag_log_relative_volume
        
        df = row.to_frame().T.drop(columns=['log_relative_volume'])
        pred_val = model.predict(df)
        ind = (features_copy.brand == brand) & (features_copy.country == country) & (features_copy.month_num == month) 
        features_copy.loc[ind, 'log_relative_volume'] = pred_val[0]
        i+=1
    return features_copy

In [ ]:
preds = get_log_relative_volume(
    pipe, 
    full_with_volume_1.loc[:, features.columns[:-1]]
)

In [ ]:
assert len(preds) == len(full_with_volume_1), 'Duplicated'

In [ ]:
assert sum(preds['log_relative_volume'].isna()) == 0, 'Missing'
assert sum(preds['log_relative_volume'].isnull()) == 0, 'Missing'

In [ ]:
features_df = preds.loc[
    :, 
    ['country', 'brand', 'month_num', 'volume_1', 'log_relative_volume']].drop_duplicates()

In [ ]:
assert len(features_df) == len(features_df.loc[:, ['country', 'brand', 'month_num']]), 'Duplicates'

In [ ]:
features_df.to_csv(os.path.join(features_path, 'feat_01.csv'), index=False)

## Add feature
$$log\Big(\frac{vol_{t} + 1}{vol_{t-1}+1}\Big)$$

* Name: relative_volume_previous

In [ ]:
train_with_predicted_log_relative_volume = train.merge(
    preds.loc[:, ['country', 'brand', 'month_num', 'volume_1', 'log_relative_volume']],
    'inner',
    on=['country', 'brand', 'month_num']
)

In [ ]:
assert len(train_with_predicted_log_relative_volume) == len(train), 'Duplicated values'

In [ ]:
volume_previous_month = train_with_predicted_log_relative_volume.copy()
volume_previous_month['previous_month'] = volume_previous_month.month_num - 1
volume_previous_month = volume_previous_month.merge(
    volume.loc[: , ['country', 'brand', 'volume', 'month_num']].rename(
        columns={'volume':'volume_lag_1', 'month_num':'previous_month'}
    ),
    'left',
    on=['country', 'brand', 'previous_month']
).merge(
    volume.loc[volume.month_num == -2, ['country', 'brand', 'volume']].rename(
        columns={'volume':'volume_2'}
    ),
    'left',
    on=['country', 'brand']
)

In [ ]:
assert len(volume_previous_month) == len(train_with_predicted_log_relative_volume), 'Duplicated values'
assert sum(volume_previous_month.volume_lag_1.isna()) == 0, 'NA values'
assert sum(volume_previous_month.volume_lag_1.isnull()) == 0, 'NA values'

In [ ]:
assert sum(volume_previous_month.volume_2.isna()) == 0, 'NA values'
assert sum(volume_previous_month.volume_2.isnull()) == 0, 'NA values'

In [ ]:
volume_previous_month['log_relative_volume_previous'] = np.log(
    (volume_previous_month.volume + 1)/(volume_previous_month.volume_lag_1 + 1)
)
volume_previous_month['log_relative_volume_1'] = np.log(
    (volume_previous_month.volume_1 + 1)/(volume_previous_month.volume_2 + 1)
)

In [ ]:
assert sum(volume_previous_month.log_relative_volume_previous.isna()) == 0, 'log_relative_volume_previous contains NA values'
assert sum(volume_previous_month.log_relative_volume_previous.isnull()) == 0, 'log_relative_volume_previous contains null values'
assert sum(volume_previous_month.log_relative_volume_previous == np.inf) == 0, 'log_relative_volume_previous contains inf values'
assert sum(volume_previous_month.log_relative_volume_previous == -np.inf) == 0, 'log_relative_volume_previous contains -inf values'

In [ ]:
assert sum(volume_previous_month.log_relative_volume_1.isna()) == 0, 'relative_volume_1 contains NA values'
assert sum(volume_previous_month.log_relative_volume_1.isnull()) == 0, 'relative_volume_1 contains null values'
assert sum(volume_previous_month.log_relative_volume_1 == np.inf) == 0, 'relative_volume_1 contains inf values'
assert sum(volume_previous_month.log_relative_volume_1 == -np.inf) == 0, 'relative_volume_1 contains -inf values'

In [ ]:
volume_previous_month['lag_log_relative_volume_previous'] = volume_previous_month.groupby(
    ['country', 'brand'])['log_relative_volume_previous'].shift(1)

In [ ]:
volume_previous_month['lag_log_relative_volume_previous'] = np.where(
    volume_previous_month.month_num == 0,
    volume_previous_month.log_relative_volume_1,
    volume_previous_month.lag_log_relative_volume_previous
)

In [ ]:
volume_previous_month

In [ ]:
cols = list(preds.columns) + ['lag_log_relative_volume_previous'] 

In [ ]:
features = volume_previous_month.loc[:, cols]
target = volume_previous_month.log_relative_volume_previous

In [ ]:
categorical_cols = ['country', 'brand', 'therapeutic_area', 'presentation', 'month_name']
te = TargetEncoder(cols=categorical_cols)
pipe2  = Pipeline([
    ("te", te),
    ("imp", SimpleImputer(strategy="mean")),
    ("sc", StandardScaler()),
    ("model", Lasso(alpha=0.001, max_iter=2000))
])

In [ ]:
pipe2.fit(features, target)

In [ ]:
def get_log_relative_volume_previous(model, features):
    features_copy = features.copy()
    features_copy.sort_values(by=['country', 'brand', 'month_num'], inplace=True)
    features_copy['log_relative_volume_previous'] = float('-inf')
    i=0
    
    for index, row in features_copy.iterrows():
        if(i%5000 == 0):
            print('Iteration:', i)
        country = row.country
        brand = row.brand
        month = row.month_num
        
        if month == 0:
            volume_1 = find_closest_volume(country, brand, 0, 1, np.mean)
            volume_2 = find_closest_volume(country, brand, -1, 1, np.mean)
            lag_log_relative_volume_previous = np.log((volume_1 + 1)/(volume_2+1))
        else:
            ind = (features_copy.country == country) &\
            (features_copy.brand == brand) &\
            (features_copy.month_num == month -1)
            lag_log_relative_volume_previous = features_copy.loc[ind, 'log_relative_volume_previous']

        row.at['lag_log_relative_volume_previous'] = lag_log_relative_volume_previous
        df = row.to_frame().T.drop(columns=['log_relative_volume_previous'])
        pred_val = model.predict(df)
        ind = (features_copy.brand == brand) & (features_copy.country == country) & (features_copy.month_num == month) 
        features_copy.loc[ind, 'log_relative_volume_previous'] = pred_val[0]
        i+=1
    return features_copy

In [ ]:
preds2 = get_log_relative_volume_previous(pipe2, preds)

In [ ]:
preds2

In [ ]:
features_df = preds2.loc[
    :, 
    ['country', 'brand', 'month_num', 'volume_1', 'log_relative_volume', 'log_relative_volume_previous']].drop_duplicates()

In [ ]:
features_df.to_csv(os.path.join(features_path, 'feat_02.csv'), index=False)